In [1]:
import json
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from nltk_utils import tokenize, stem, bag_of_words
from model import NeuralNet

In [2]:
# Open file json
with open('intents.json', 'r') as f:
    intents = json.load(f)

In [3]:
# Define list
all_words = []  # Setence
tags = []       # Tags of setence
xy = []         # X and Y of setence

In [4]:
# Getting setence, tags, xy
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

In [5]:
# Stemming
ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words] 

In [6]:
# Remove duplicate word
all_words = sorted(set(all_words))      
tags = sorted(set(tags)) 
# print(tags)

In [7]:
# Define train data
x_train = []
y_train = []
for (pattern_setence, tag) in xy:
    # Getting the feature
    bag = bag_of_words(pattern_setence, all_words)
    x_train.append(bag)
    
    # Getting label from index of tag
    label = tags.index(tag)
    y_train.append(label)   # CrossEntropyLoss

In [8]:
# Convert to array
x_train = np.array(x_train)
y_train = np.array(y_train)

In [9]:
# Create custom class dataset
class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(x_train)
        self.x_data = x_train
        self.y_data = y_train
    
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
    
    def __len__(self):
        return self.n_samples

In [10]:
# Define Hyperparameter
batch_size = 8
hidden_size = 8
input_size = len(x_train[0])
output_size = len(tags)
learning_rate = 0.001
num_epochs = 1000
# print(input_size, len(all_words))
# print(output_size, tags)

In [11]:
# Create dataset
dataset = ChatDataset()
# train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=2)
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0)
# train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, num_workers = 2, persistent_workers=True)

In [18]:
for (words, labels) in train_loader:
    first_word = words
    first_label = labels
    print(f'words \t: {words}')
    print(f'labels \t: {labels}')
    if first_word != None and first_label != None:
        break

words 	: tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.

In [31]:
all_words[0:5]

["'s", 'a', 'accept', 'anyon', 'are']

In [32]:
xy[0:3]

[(['Hi'], 'greeting'),
 (['Hey'], 'greeting'),
 (['How', 'are', 'you'], 'greeting')]

In [25]:
len(tags)

7

In [20]:
first_word.shape

torch.Size([8, 54])

In [22]:
first_label

tensor([4, 4, 6, 3, 5, 3, 0, 0], dtype=torch.int32)

In [39]:
x_train.shape

(26, 54)

In [34]:
y_train.shape

(26,)

In [12]:
# Checking cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
torch.cuda.is_available()

False

In [13]:
# Create model
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# Training model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device, dtype=torch.int64)
        
        # Forward
        outputs = model(words)
        loss = criterion(outputs, labels)
        
        # Backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Print loss per 100 epoch
    if (epoch + 1) % 100 == 0:
        print(f'epoch : {epoch+1}/{num_epochs}, loss={loss.item():.4f}')

# Print final loss when training done
print(f'final loss, loss={loss.item():.4f}')

epoch : 100/1000, loss=1.8516
epoch : 200/1000, loss=0.4274
epoch : 300/1000, loss=0.0888
epoch : 400/1000, loss=0.0413
epoch : 500/1000, loss=0.0070
epoch : 600/1000, loss=0.0037
epoch : 700/1000, loss=0.0035
epoch : 800/1000, loss=0.0021
epoch : 900/1000, loss=0.0001
epoch : 1000/1000, loss=0.0004
final loss, loss=0.0004


In [15]:
# Save data
data = {
    'model_state' : model.state_dict(),
    'input_size' : input_size,
    'output_size' : output_size,
    'hidden_size' : hidden_size,
    'all_words' : all_words,
    'tags' : tags,
}

# Save file of data
FILE = 'data.pth'
torch.save(data, FILE)
print(f'Training complete. file saved to {FILE}')

Training complete. file saved to data.pth
